In [1]:
%load_ext autoreload
%autoreload 2

## Data

In [2]:
from utils import *

data = load_data()
len(data)

Loading environment variables from c:\Users\roman\Documents\dev\AutoTOD\.env


9906

In [3]:
dialog, dialog_id = pick_dialog(data, dialog_id='random', domain='all', exclusive=False)
print_dialog_goal(dialog, dialog_id)
print()
show_dialog_text(dialog)

[Dialog Id] PMUL2446.json

[User Goals]
You are looking for information in Cambridge.
You are looking for a restaurant. The restaurant should serve chinese food and should be in the expensive price range.
Once you find the restaurant you want to book a table for 3 people at 11:45 on wednesday.
Make sure you get the reference number.
You are also looking for places to go in town. The attraction should be in the east.
Make sure you get address, postcode, and entrance fee.
You also want to book a taxi to commute between the two places.
You want to make sure it arrives the restaurant by the booked time.
Make sure you get contact number and car type.

[restaurant]
{'book': {'day': 'wednesday', 'invalid': False, 'people': '3', 'time': '11:45'},
 'fail_book': {},
 'fail_info': {},
 'info': {'food': 'chinese', 'pricerange': 'expensive'}}

[attraction]
{'fail_info': {'area': 'south'},
 'info': {'area': 'east'},
 'reqt': ['address', 'postcode', 'entrance fee']}

[taxi]
{'fail_info': {}, 'info': 

## Engine

In [4]:
import engine

model_name = 'gpt-3.5-turbo'
run_result = engine.run(dialog, agent_type='react', agent_model=model_name, user_model=model_name)

================================================== Turn 1 ==================================================

User: Hello, I'm looking for a restaurant in Cambridge that serves high Chinese food and is more expensive than it's competitors. 


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Restaurant Query
Action Input: area: Cambridge, food: Chinese, price: high

> Entering new SQLDatabaseChainWithCleanSQL chain...
area: Cambridge, food: Chinese, price: high
SQLQuery:SELECT "name", "address", "phone", "signature"
FROM restaurant
WHERE "area" != "Cambridge" AND "food" = "chinese" AND "pricerange" = "expensive"
LIMIT 5;
```

Warning: You can only execute one statement at a time.

## Eval

In [7]:
from utils import DOMAINS
from evaluate import evaluate_by_domain

for domain in DOMAINS:
    if not dialog['goal'].get(domain):
        continue
    print('=' * 40)
    try:
        eval_result = evaluate_by_domain(domain, run_result)
    except Exception as e:
        msg = f'Run dialog failed as {e.__class__.__name__}: '
        print(colored(msg, 'red') + str(e))

[domain] attraction
[goal]
    info: {'type': 'cinema', 'area': 'centre'}
    reqt: ['address', 'postcode', 'phone']
    fail_info: {'type': 'entertainment', 'area': 'centre'}
[inform] complete: 1
    venue_name: Vue Cinema
    venue_info: Attraction:
{'address': 'the grafton centre, east road',
 'area': 'centre',
 'entrance_fee': '?',
 'id': 75,
 'name': 'vue cinema',
 'phone': '08712240240',
 'postcode': 'cb11ps',
 'type': 'cinema'}
[success] complete: 1
    slot_values: {'address': 'The Grafton Centre, East Road', 'postcode': 'CB1 1PS', 'phone': '0871 224 0240'}
[book] complete: None
    refer_number: None
    book_record: None
[domain] train
[goal]
    info: {'leaveAt': '13:45', 'destination': 'peterborough', 'day': 'sunday', 'departure': 'cambridge'}
    reqt: ['trainID', 'duration', 'price']
    fail_info: {}
[inform] complete: 0
[success] complete: 0
    slot_values: {'trainID': 'TR6028', 'duration': '51 minutes', 'price': '£23.60'}
[book] complete: None
    refer_number: None
 